In [1]:
import os
os.chdir('/content/drive/MyDrive/MVA/KKML')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/MVA/KKML'

# Kernel Methods: Challenge

Julia Linhart, Roman Castagné, Louis Bouvier

Preliminary functions:

In [1]:
def write_csv(ids, labels, filename):
    """
    inputs:
        - ids: list of ids, should be an increasing list of integers
        - labels: list of corresponding labels, either 0 or 1
        - file: string containing the name that should be given to the submission file    
    """
    df = pd.DataFrame({"Id": ids, "Bound": labels})
    df["Bound"] = df["Bound"].replace([-1], 0)
    df.to_csv(filename, sep=',', index=False)

# I) Preprocessing

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from functools import partial
from scipy.spatial import distance_matrix
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import GridSearchCV
import cvxpy as cp
import warnings
import time
from itertools import product
from numba import jit


warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
data_folder = 'data' # 'machine-learning-with-kernel-methods-2021'

X_train_1 = pd.read_csv(f'{data_folder}/Xtr2_mat100.csv', sep = ' ', index_col=False, header=None)
y_train_1 = pd.read_csv(f'{data_folder}/Ytr2.csv')

In [33]:
y_train_1.describe()

,Id,Bound
count,2000.000000,2000.000000
mean,4999.500000,0.498500
std,577.494589,0.500123
min,4000.000000,0.000000
25%,4499.750000,0.000000
50%,4999.500000,0.000000
75%,5499.250000,1.000000
max,5999.000000,1.000000


In [34]:
y_train_1 = np.array(y_train_1)[:,1]

In [35]:
X_train_1.describe()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,0.010565,0.010201,0.010375,0.011587,0.011609,0.010707,0.009359,0.011957,0.009571,0.010582,...,0.007951,0.009457,0.008554,0.009283,0.008261,0.009614,0.011141,0.009777,0.008217,0.008565
std,0.012278,0.010723,0.011467,0.011453,0.012182,0.010478,0.009789,0.012444,0.013805,0.013652,...,0.009605,0.009701,0.009350,0.009741,0.012341,0.010338,0.010863,0.010402,0.009709,0.009283
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.010870,0.010870,0.010870,0.010870,0.010870,0.010870,0.010870,0.010870,0.000000,0.010870,...,0.010870,0.010870,0.010870,0.010870,0.000000,0.010870,0.010870,0.010870,0.010870,0.010870
75%,0.010870,0.021739,0.021739,0.021739,0.021739,0.021739,0.010870,0.021739,0.010870,0.021739,...,0.010870,0.010870,0.010870,0.010870,0.010870,0.010870,0.021739,0.010870,0.010870,0.010870
max,0.086957,0.065217,0.097826,0.065217,0.065217,0.054348,0.054348,0.076087,0.097826,0.184783,...,0.054348,0.065217,0.054348,0.054348,0.086957,0.065217,0.076087,0.065217,0.065217,0.043478


In [36]:
X_train_1 = np.array(X_train_1)
print(X_train_1.shape)
X_train_1 = (X_train_1 - X_train_1.mean(axis=0))/X_train_1.std(axis=0)

(2000, 100)


In [37]:
print(y_train_1.shape)

(2000,)


# II) First linear models of the mat100 input

## A) Logistic regression

In [38]:
def g(z):
    """
    input:
    - z (any size): an array-like element
    ouput:
    - the element-wize application of the sigmoïd function on z
    """
    return 1/(1+np.exp(-z))

In [39]:
def compute_loss(X,y,w,b):
    """
    inputs:
    - X (size: Nxd): the points we want to classify
    - y (size: Nx1): the values of the classes
    - w (size: 1xd): the weights of the affine mapping of x
    - b (size: 1x1): the constant of the affine mapping of x
    output:
    - the opposite of the log-likelihood of the Logistic Regression model computed with respect to
    the points (X,y) and the parameters w,b
    """
    X_tilde = np.hstack([X, np.ones((X.shape[0], 1))])
    w_tilde = np.hstack((w,b))
    return -np.sum(y * np.log(g(w_tilde@X_tilde.T)) + (1-y) * np.log(1-g(w_tilde@X_tilde.T)), axis=1)

In [40]:
def compute_grad(X,y,w,b):
    """
    inputs:
    - X (size: Nxd): the points we want to classify
    - y (size: Nx1): the values of the classes
    - w (size: 1xd): the weights of the affine mapping of x
    - b (size: 1x1): the constant of the affine mapping of x
    output:
    - the gradient of the loss of the Logistic Regression model computed 
    with respect to (w,b) = w_tilde having the points (X,y) 
    """
    X_tilde = np.hstack([X, np.ones((X.shape[0], 1))])
    w_tilde = np.hstack((w,b))
    return -X_tilde.T @ (y - g(w_tilde@X_tilde.T).reshape(-1,))

In [41]:
def compute_hess(X,y,w,b):
    """
    inputs:
    - X (size: Nxd): the points we want to classify
    - y (size: Nx1): the values of the classes
    - w (size: 1xd): the weights of the affine mapping of x
    - b (size: 1x1): the constant of the affine mapping of x
    output:
    - the hessian of the loss of the Logistic Regression model computed 
    with respect to (w,b) = w_tilde having the points (X,y) 
    """
    X_tilde = np.hstack([X, np.ones((X.shape[0], 1))])
    w_tilde = np.hstack((w,b))    
    temp = (g(w_tilde @ X_tilde.T) * (g(w_tilde @ X_tilde.T) - 1)).reshape(-1,)
    return -X_tilde.T @ np.diag(temp) @ X_tilde

In [42]:
def backtracking(X,y,w,b,delta,grad,alpha=0.1,beta=0.7):
    """
    inputs:
    - X (size: Nxd): the points we want to classify
    - y (size: Nx1): the values of the classes
    - w (size: 1xd): the weights of the affine mapping of x
    - b (size: 1x1): the constant of the affine mapping of x
    - delta (size n): direction of the search
    - grad (size n): value of the gradient at point (w,b)
    - alpha: factor of the slope of the line in the backtracking line search
    - beta: factor of reduction of the step length
    
    outputs:
    - t: the step length for the Newton step on the objective function
    computed with backtracking line search towards delta"""
        
    t = 1
    while(compute_loss(X, y, w+t*delta[:-1], b+t*delta[-1])>
            compute_loss(X,y,w,b) + alpha*t*grad.T @ delta):
        t = beta*t
    return t

In [43]:
def Newton(X, y, w0, b0, eps=pow(10,-1)):
    """
    inputs:
    - X (size: Nxd): the points we want to classify
    - y (size: Nx1): the values of the classes
    - w0 (size: 1xd): the initial weights of the affine mapping of x
    - b0 (size: 1x1): the initial constant of the affine mapping of x
    output:
    - the paramer vector w_tilde_hat = (w_hat, b_hat) which maximizes the log-likelihood of 
    the sample (X,y) in the Logistic Regression model (or minimizes the loss)
    - the cached values of the loss evaluated along training
    """
    w_, b_ = w0, b0
    grad = compute_grad(X, y, w0, b0)
    hess = compute_hess(X, y, w0, b0)
    
#     inv_hess = np.linalg.inv(compute_hess(X,y,w0,b0))
    inv_hess, _, _, _ = np.linalg.lstsq(hess, np.eye(hess.shape[0]))
    dec_2 = grad.T@inv_hess@grad
    Loss_hist = [compute_loss(X,y,w0,b0)]
    while dec_2/2 > eps: # condition on the Newton decrement
        grad = compute_grad(X,y,w_,b_)
        hess = compute_hess(X,y,w_,b_)
        
#         inv_hess = np.linalg.inv(compute_hess(X,y,w_,b_))
        inv_hess, _, _, _ = np.linalg.lstsq(hess, np.eye(hess.shape[0]))
        dec_2 = grad.T@inv_hess@grad
        delta = - inv_hess@grad
        t_bt = backtracking(X, y, w_, b_, delta, grad)
        w_ = w_ + t_bt*delta[:-1]
        b_ = b_ + t_bt*delta[-1]
        Loss_hist.append(compute_loss(X,y,w_,b_))
    return w_, b_, Loss_hist

In [44]:
def predict_LogReg(x,w,b):
    """
    inputs:
    - x (size 1xd): a point in R^d
    - w (size: 1xd): the weights of the affine mapping of x
    - b (size: 1x1): the constant of the affine mapping of x
    output:
     - the predicted class for the associated y given the
    Logistic Regression parameters
    """    
    return (w.T@x + b > 0).astype("int")

In [45]:
class LogisticRegressor(BaseEstimator, ClassifierMixin):
    
    def __init__(self, lamb=1.):
        """
        This class implements methods for fitting and predicting with a LogesticRegression for classification 
        inputs:
        - lamb : the regularisation parameter
        """
        self.lamb = lamb
    
    def fit(self, X, y):
        """
        inputs:
        - X (size: Nxd): the points we want to classify
        - y (size: Nx1): the values of the classes
        outputs:
        - the value of MLE estimation (w_hat, b_hat) in the Linear regression model
        """
        w0, b0 = np.random.randn(1, 100)*0.07, np.zeros((1,1))
        self.w_, self.b_, _ = Newton(X, y, w0, b0)
        
        return self
        
    def predict(self, X):
        """
        inputs:
        - X (size Nxd): a point in R^d
        - w (size: 1xd): the weights of the affine mapping of x
        - b (size: 1x1): the constant of the affine mapping of x
        output:
         - the predicted class for the associated y given the
        Linear Regression parameters
        """    
        return (self.w_@X.T + self.b_ > 1/2).astype("int")
        
    def score(self, X, y):
        """
        inputs:
        - X (size Nxd): the points in R^d we want to classify
        - y (size Nx1): the labels of the points
        """
        y_pred = self.predict(X)
        return np.sum(y_pred == y)/y.shape[0]

In [46]:
dim = 100
Nb_samples = 2000
prop_test = 0.05

all_y_eval = []

np.random.seed(1)
for name in [0, 1, 2]:
    X = pd.read_csv(f'{data_folder}/Xtr{name}_mat100.csv', sep = ' ', index_col=False, header=None).to_numpy()
    y = pd.read_csv(f'{data_folder}/Ytr{name}.csv')
    y = y["Bound"].to_numpy()
    
    mean, std = X.mean(axis=0), X.std(axis=0)
    X = (X - mean)/std

    tr_indices = np.random.choice(Nb_samples, size=int((1-prop_test)*Nb_samples), replace=False)
    te_indices = [idx for idx in range(Nb_samples) if idx not in tr_indices]

    X_tr = X[tr_indices]
    X_te = X[te_indices]
    
    y_tr = y[tr_indices]
    y_te = y[te_indices]
    
    assert X_tr.shape[0] + X_te.shape[0] == X.shape[0]
    assert y_tr.shape[0] + y_te.shape[0] == y.shape[0]
    
    # Fitting
    logreg = LogisticRegressor()
    
    logreg.fit(X_tr, y_tr)

    
    print(f"Accuracy on train set {name}: {logreg.score(X_tr, y_tr):.2f}")
    print(f"Accuracy on test set {name} : {logreg.score(X_te, y_te):.2f}")   
    
    # Prediction on the new set
    X_eval = pd.read_csv(f'{data_folder}/Xte{name}_mat100.csv', sep = ' ', index_col=False, header=None).to_numpy()
    X_eval = (X_eval - mean)/std
    y_eval = logreg.predict(X_eval)
    all_y_eval.append(y_eval)
    
all_y_eval = np.hstack(all_y_eval).reshape(-1)

/Users/louisbouvier/Documents/MVA-MPRO/MVA/S2/Learning/Kernel Methods/challenge/KM_env/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
/Users/louisbouvier/Documents/MVA-MPRO/MVA/S2/Learning/Kernel Methods/challenge/KM_env/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


Accuracy on train set 0: 0.62
Accuracy on test set 0 : 0.56
Accuracy on train set 1: 0.60
Accuracy on test set 1 : 0.59
Accuracy on train set 2: 0.70
Accuracy on test set 2 : 0.66


In [47]:
ids = np.arange(all_y_eval.shape[0])
filename = "results/submission_log_reg.csv"

# write_csv(ids, all_y_eval, filename)

## B) Ridge regression

In [48]:
def compute_RR_MLE(X,y,lamb):
    """
    inputs:
    - X (size: Nxd): the points we want to classify
    - y (size: Nx1): the values of the classes
    outputs:
    - the value of MLE estimation (w_hat, b_hat) in the Linear regression model
    """
    X_tilde = np.vstack((X,np.ones(X.shape[1])))
    temp = np.linalg.inv(X_tilde@X_tilde.T + lamb*X.shape[1]*np.eye(1+X.shape[0]))@X_tilde@y.T
    return temp[:-1], temp[-1]

In [49]:
def predict_RR(x,w,b):
    """
    inputs:
    - x (size 1xd): a point in R^d
    - w (size: 1xd): the weights of the affine mapping of x
    - b (size: 1x1): the constant of the affine mapping of x
    output:
     - the predicted class for the associated y given the
    Linear Regression parameters
    """    
    return (w.T@x+b>1/2).astype("int")

In [50]:
class RidgeRegressor(BaseEstimator, ClassifierMixin):
    
    def __init__(self, lamb=1.):
        """
        This class implements methods for fitting and predicting with a RidgeRegressor used for classification 
        (by thresholding the value regressed).
        inputs:
        - lamb : the regularisation parameter
        """
        self.lamb = lamb
    
    def fit(self, X, y):
        """
        inputs:
        - X (size: Nxd): the points we want to classify
        - y (size: Nx1): the values of the classes
        outputs:
        - the value of MLE estimation (w_hat, b_hat) in the Linear regression model
        """
        X_tilde = np.hstack((X, np.ones((X.shape[0], 1))))
        temp = np.linalg.inv(X_tilde.T @ X_tilde + self.lamb * X.shape[0] * np.eye(X_tilde.shape[1])) @ (X_tilde.T @ y)
        self.w_ = temp[:-1]
        self.b_ = temp[-1]

        return self
        
    def predict(self, X):
        """
        inputs:
        - x (size Nxd): a point in R^d
        - w (size: 1xd): the weights of the affine mapping of x
        - b (size: 1x1): the constant of the affine mapping of x
        output:
         - the predicted class for the associated y given the
        Linear Regression parameters
        """    
        return (self.w_@X.T + self.b_ > 1/2).astype("int")
        
    def score(self, X, y):
        """
        inputs:
        - X (size Nxd): the points in R^d we want to classify
        - y (size Nx1): the labels of the points
        """
        y_pred = self.predict(X)
        return np.sum(y_pred == y)/y.shape[0]

In [51]:
dim = 100
Nb_samples = 2000
prop_test = 0.05
lamb = 0.1

all_y_eval = []

np.random.seed(1)
for name in [0, 1, 2]:
    # Data processing
    X = pd.read_csv(f'{data_folder}/Xtr{name}_mat100.csv', sep = ' ', index_col=False, header=None).to_numpy()
    y = pd.read_csv(f'{data_folder}/Ytr{name}.csv')
    y = y["Bound"].to_numpy()
    
    mean, std = X.mean(axis=0), X.std(axis=0)
    X = (X - mean)/std

    tr_indices = np.random.choice(Nb_samples, size=int((1-prop_test)*Nb_samples), replace=False)
    te_indices = [idx for idx in range(Nb_samples) if idx not in tr_indices]

    X_tr = X[tr_indices]
    X_te = X[te_indices]
    
    y_tr = y[tr_indices]
    y_te = y[te_indices]
    
    assert X_tr.shape[0] + X_te.shape[0] == X.shape[0]
    assert y_tr.shape[0] + y_te.shape[0] == y.shape[0]
    
    # Fitting the classifier
    params = {'lamb': np.linspace(0.001, 0.1, 20)}
    rr = GridSearchCV(RidgeRegressor(), params)
#     rr = RidgeRegressor(lamb=lamb)

    rr.fit(X_tr, y_tr)
    
    print(rr.best_params_)
    
    print(f"Accuracy on train set {name}: {rr.score(X_tr, y_tr):.2f}")
    print(f"Accuracy on test set {name} : {rr.score(X_te, y_te):.2f}")
    
    # Prediction on the new set
    X_eval = pd.read_csv(f'{data_folder}/Xte{name}_mat100.csv', sep = ' ', index_col=False, header=None).to_numpy()
    X_eval = (X_eval - mean)/std
    y_eval = rr.predict(X_eval)
    all_y_eval.append(y_eval)
    
all_y_eval = np.hstack(all_y_eval).reshape(-1)

{'lamb': 0.021842105263157895}
Accuracy on train set 0: 0.65
Accuracy on test set 0 : 0.60
{'lamb': 0.001}
Accuracy on train set 1: 0.64
Accuracy on test set 1 : 0.57
{'lamb': 0.04268421052631579}
Accuracy on train set 2: 0.73
Accuracy on test set 2 : 0.69


# III) Kernel baselines 

## A) Kernels

### a) Gaussian Kernel

In [52]:
def Gaussian_kernel(X1, X2, sig):
    """inputs:
    - X1 (size N1xd): a set of points
    - X2 (size N2xd): another one  
    - sig (float): the std of the kernel
    ouput:
    - the associated (N1xN2) Gaussian kernel
    """
    return np.exp(-distance_matrix(X1,X2)/(2*sig**2))

### b) Spectrum kernel

In [76]:
def spectrum(x,k):
    l = len(x)
    spectrum_x = np.array([x[i:(i + k)] for i in range(l - k + 1)])
    return np.array(spectrum_x)

def Spectrum_kernel(X1, X2, k):
    """inputs:
    - X1 (size N1xd): a set of sequences
    - X2 (size N2xd): another one  
    - k (int): the length of the substrings 
    ouput:
    - the associated (N1xN2) Spectrum kernel
    """
    # substrings: all possible combinations of A,T,G,C of length k
    A_k = [''.join(s) for s in product(["A", "T", "G", "C"], repeat=k)]

    # nb of occurances of the elements of A_k in the k-spectrum of X1 (resp. X2)
    phi_spect_X1 = np.array([[np.sum(spectrum(x,k)==u) for u in A_k] for x in X1])
    phi_spect_X2 = np.array([[np.sum(spectrum(x,k)==u) for u in A_k] for x in X2])
    
    K_s = phi_spect_X1 @ phi_spect_X2.T
    K_s = K_s + np.eye(K_s.shape[0], K_s.shape[1])*pow(10,-12)
    return K_s

### c) Substring kernel

In [161]:
@jit(nopython=True)
def substring_similarity(seq_1, seq_2, k, lambd):
    # Initialize K and B
    K_temp = [np.ones((len(seq_1), len(seq_2)))]
    K_temp += [np.zeros((len(seq_1), len(seq_2))) for _ in range(k)]

    B_temp = [np.ones((len(seq_1), len(seq_2)))]
    B_temp += [np.zeros((len(seq_1), len(seq_2))) for _ in range(k)]

    for l in range(1, k+1):
        # First, loop over the first sequence
        for i_str_1 in range(len(seq_1)):

            # Then, loop over the second sequence
            for i_str_2 in range(len(seq_2)):
                a = seq_1[i_str_1]
                b = seq_2[i_str_2]
                
                # If min < l, then the matrix already has zeros in the right place : we can continue
                if min(i_str_1, i_str_2) >= l-1:

                    if i_str_1 == 0 and i_str_2 == 0:
                        continue
                    
                    if i_str_2 == 0:
                        # Computation of B
                        B_temp[l][i_str_1, i_str_2] = lambd * B_temp[l][i_str_1-1, i_str_2]
                        K_temp[l][i_str_1, i_str_2] = K_temp[l][i_str_1-1, i_str_2]
                        
                    elif i_str_1 == 0:
                        B_temp[l][i_str_1, i_str_2] = lambd * B_temp[l][i_str_1, i_str_2-1]
                        K_temp[l][i_str_1, i_str_2] = K_temp[l][i_str_1, i_str_2-1]

                    else:
                        B_temp[l][i_str_1, i_str_2] = lambd * B_temp[l][i_str_1-1, i_str_2] \
                                                    + lambd * B_temp[l][i_str_1, i_str_2-1] \
                                                    - lambd ** 2 * B_temp[l][i_str_1-1, i_str_2-1] \
                                                    + lambd ** 2 * int(a == b) * B_temp[l-1][i_str_1-1, i_str_2-1]

                        # This corresponds to the sum in the computation of K
                        # !!!!! This one is probably wrong !!!!!
#                         K_sum_1 = 0
#                         for j_prime in range(i_str_2+1):
#                             if seq_2[j_prime] == a:
#                                 K_sum_1 += B_temp[l-1][i_str_1-1, j_prime-1]

#                         # Computation of K
#                         K_temp[l][i_str_1, i_str_2] = K_temp[l][i_str_1-1, i_str_2] + lambd ** 2 * K_sum_1

                        # This one is wrong too, but less
                        K_temp[l][i_str_1, i_str_2] = K_temp[l][i_str_1-1, i_str_2] \
                                                    + K_temp[l][i_str_1, i_str_2-1] \
                                                    - K_temp[l][i_str_1-1, i_str_2-1] \
                                                    + lambd ** 2 * int(a == b) * B_temp[l-1][i_str_1-1, i_str_2-1]

    return K_temp[k][-1, -1]

In [162]:
def substring_kernel(X1, X2, k, lambd):
    """
    
    """
    assert all([type(x) == str for x in X1]), "not a list of strings"
    assert all([type(x) == str for x in X2]), "not a list of strings"
    
    K = - np.ones((len(X1), len(X2)))
    
    for i, seq_1 in enumerate(X1):
        for j, seq_2 in enumerate(X2):
            # This basically corresponds to filling the lower diagonal
            if seq_2 in X1 and seq_1 in X2 and K[np.where(X1 == seq_2), np.where(X2 == seq_1)] != -1:
                K[i, j] = K[np.where(X1 == seq_2), np.where(X2 == seq_1)]
            else:
                K[i, j] = substring_similarity(seq_1, seq_2, k, lambd)

    return K

In [163]:
# Run similarity between two strings
t0 = time.time()
# k = substring_similarity("ATGCATGATGCATG", "ATGCATCATGATGT", 3, 1.)
# k = substring_similarity("ATGC", "ATGC", 3, 0.7)
# k = substring_similarity("ATTTGCCGTATC", "AGGCTCTCCAGC", 2, 0.7)
k = substring_similarity("cat", "cat", 2, 0.7)
# k = substring_similarity("ACGT", "TGCG", 2, 0.7)
k_expected = 2 * (0.7 ** 4) + 0.7 ** 6
print(f"Time to compute : {time.time() - t0:.4f}s")
print(f"Value : {k}")
print(f"Expected value : {k_expected}")

Time to compute : 1.7824s
Value : 0.24009999999999992
Expected value : 0.5978489999999999


In [178]:
# Run kernel computation between N strings
X = pd.read_csv(f'{data_folder}/Xtr0.csv', sep = ',').to_numpy()
X = X[:10,1]
t0 = time.time()
K = substring_kernel(X, X, k=5, lambd=0.7)
print(f"Time to compute K : {time.time() - t0:.2f}s")
assert np.allclose(K, K.T)

Time to compute K : 0.68s


In [179]:
k = 4
lambd = 0.7

for name in {0, 1, 2}:
    t0 = time.time()
    X_tr = pd.read_csv(f'{data_folder}/Xtr{name}.csv', sep = ',').to_numpy()
    X_tr = X_tr[:,1]
    K = substring_kernel(X_tr, X_tr, k=k, lambd=lambd)
    print(f"Computed kernel for dataset {name} in {time.time() - t0:.2f}s.")
    with open(f"{data_folder}/Ktrtr{name}_k={k}_lambda={lambd}.npy", "wb") as f:
        np.save(f, K)
    
    
    t0 = time.time()
    X_ev = pd.read_csv(f'{data_folder}/Xte{name}.csv', sep = ',').to_numpy()
    X_ev = X_ev[:, 1]
    K = substring_kernel(X_tr, X_ev, k=k, lambd=lambd)
    print(f"Computed evaluation kernel for dataset {name} in {time.time() - t0:.2f}s.")
    with open(f"{data_folder}/Ktreval{name}_k={k}_lambda={lambd}.npy", "wb") as f:
        np.save(f, K)

Computed kernel for dataset 0 in 39224.60s.
Computed evaluation kernel for dataset 0 in 91215.05s.


KeyboardInterrupt: 

In [373]:
def load_precomputed_kernel(df_train, df_eval, kernel_filename_train, kernel_filename_eval, normalize=False):
    with open(kernel_filename_train, "rb") as f:
        K_tr = np.load(f)
    with open(kernel_filename_eval, "rb") as f:
        K_ev = np.load(f)
        
    def precomputed_kernel(X1, X2, **args):
        K = np.zeros((len(X1), len(X2)))
        
        idx1 = []
        for x in X1: # Needed to get elements in the right order
            idx1.append(df_train[df_train['seq'] == x]['Id'].iloc[0])

        if sum(df_train['seq'].isin(X2)) >= len(X2): # Check if all elements are in the training set
            idx2 = []
            for x in X2: # Needed to get elements in the right order
                idx2.append(df_train[df_train['seq'] == x]['Id'].iloc[0])
            # Extract submatrix using correct indices
            K = K_tr[np.ix_(idx1, idx2)]
                                        
        elif sum(df_eval['seq'].isin(X2)) >= len(X2): # Check if all elements are in the evaluation set
            idx2 = []
            for x in X2: # Needed to get elements in the right order
                idx2.append(df_eval[df_eval['seq'] == x]['Id'].iloc[0])
            # Extract submatrix using correct indices
            K = K_ev[np.ix_(idx1, idx2)]
        
        # This is to have a positive matrix 'à la bled'
        K += 1e-8
        # Divide all elements by row and column values
        if normalize:
            diag = np.copy(np.diag(K))
            K = K / diag[:, None]
            K = K / diag
        return K
    

    return precomputed_kernel

In [374]:
precomputed_kernel = load_precomputed_kernel(df, df_eval, 
                                             f"{data_folder}/Ktrtr{name}_k=4_lambda=0.7.npy",
                                             f"{data_folder}/Ktreval{name}_k=4_lambda=0.7.npy",
                                             normalize=True)

X1 = np.array(['TCCTGTGCACATCTGCACCCCTGTTGTGGCCACAAAATGATCCGGCACCACCCAGTGGGAGACGACAGAGGTGGCAATGGGGTGTCGGCTCTGACGCCTCC',
               'TGAAATCCTGCCTCTTCCATGAAGCCGCCTCTGAGAACTGCTGCCAATAGGTGGCTCTTCTTTCTCAGAAGTTCTATTGCATGAATTATTTTCAGTATAAC'])
X2 = np.array(['CATCCTACATGATGCCATTGGCATCAGCACCCTAAATCAAATCAAACCAGATGGCTTCACATCCAGCCTCAAAAACTTCCTGTGGAATCACAAGACGTGAA',
               'TGAAATCCTGCCTCTTCCATGAAGCCGCCTCTGAGAACTGCTGCCAATAGGTGGCTCTTCTTTCTCAGAAGTTCTATTGCATGAATTATTTTCAGTATAAC'])
precomputed_kernel(X2, X2)

array([[0.00025907, 0.00018632],
       [0.00018632, 0.00029442]])

In [365]:
2442.6139 / (3859.889 ** 2)

0.0001639476487958876

### d) Mismatch kernel

In [ ]:
class Node(object):
    def __init__(self, parent, letter, m=1):
        self.m = m
        self.parent = parent
        self.letter = letter
        self.sequence = self.parent.get_sequence()+self.letter
        self.pointers = []
        self.mismatch_per_pointer = []
        
    def get_sequence():
        return self.sequence
    
    def get_pointers():
        return self.pointers

In [ ]:
class Tree(Node):
    def __init__(self,k,m):
        self.root

## B) Algorithms
### 1. Kernel Ridge Regression

In [58]:
def compute_KRR_MLE(X, y, lamb, sig=10):
    """
    inputs:
    - X (size: N_trxd): the points of the training set
    - y (size: N_trx1): the values of the classes
    outputs:
    - the value of MLE estimation (w_hat, b_hat) in the kernel ridge regression model
    """
    K = Gaussian_kernel(X, X, sig=sig)
    alpha = np.linalg.inv(K+lamb*X.shape[1]*np.eye(X.shape[1]))@y.T
    return alpha

In [59]:
def predict_KRR(X_tr, X_te, alpha, sig=10):
    """
    inputs:
    - X_tr (size N_trxd): the points of the training set
    - X_te (size N_texd): the points of the test set we want to classify
    - w (size: 1xd): the weights of the affine mapping 
    - b (size: 1x1): the constant of the affine mapping
    output:
     - the predicted class for the associated y_te given the
    Linear Regression parameters
    """    
    K_te_tr = Gaussian_kernel(X_tr, X_te, sig=sig)
    return 2*(alpha.T@K_te_tr>0).astype("int")-1

In [60]:
class KernelRidgeRegressor(BaseEstimator, ClassifierMixin):
    
    def __init__(self, lamb=1., sigma=1., kernel='gaussian'):
        """
        This class implements methods for fitting and predicting with a KernelRidgeRegressor used for classification 
        (by thresholding the value regressed). Any kernel can be used. 
        inputs:
        - lamb : the regularisation parameter 
        - sigma : the parameter of the Gaussian kernel (if Gaussian kernel selected)
        - kernel : the kernel we consider
        """
        self.lamb = lamb
        self.sigma = sigma
        self.kernel = kernel
        if self.kernel == 'gaussian':
            self.kernel_ = partial(Gaussian_kernel, sig=sigma)
        else:
            raise NotImplementedError(f"Kernel {self.kernel} is not implemented yet")
    
    def fit(self, X, y):
        """
        inputs:
        - X (size: N_trxd): the points of the training set
        - y (size: N_trx1): the values of the classes
        """
        # We keep values of training in memory for prediction
        self.X_tr_ = np.copy(X)
        K = self.kernel_(X, X, sig=self.sigma)
        self.alpha_ = np.linalg.inv(K+self.lamb*X.shape[0]*np.eye(X.shape[0]))@y
        
        return self
        
    def predict(self, X):
        """
        inputs:
        - X (size N_texd): the points in R^d we want to classify
        output:
         - the predicted class for the associated y given the
        Linear Regression parameters
        """
        K_tr_te = self.kernel_(self.X_tr_, X, sig=self.sigma)
        
        return 2 * (self.alpha_.T@K_tr_te > 0).reshape(-1, ).astype("int") - 1
        
    def score(self, X, y):
        """
        inputs:
        - X (size N_texd): the points in R^d we want to classify
        - y (size N_tex1): the labels of the points
        """
        y_pred = self.predict(X)
        
        return np.sum(y_pred == y)/y.shape[0]

In [61]:
dim = 100
Nb_samples = 2000
prop_test = 0.2
lamb = 0.5
sigma = 1.2

all_y_eval = []

np.random.seed(1)
for name in [0, 1, 2]:
    # Data Processing
    X = pd.read_csv(f'{data_folder}/Xtr{name}_mat100.csv', sep = ' ', index_col=False, header=None).to_numpy()
    y = pd.read_csv(f'{data_folder}/Ytr{name}.csv')
    y = y["Bound"].to_numpy()
    y[y==0] = -1
    
    mean, std = X.mean(axis=0), X.std(axis=0)
    X = (X - mean)/std

    tr_indices = np.random.choice(Nb_samples, size=int((1-prop_test)*Nb_samples), replace=False)
    te_indices = [idx for idx in range(Nb_samples) if idx not in tr_indices]

    X_tr = X[tr_indices]
    X_te = X[te_indices]
    
    y_tr = y[tr_indices]
    y_te = y[te_indices]
    
    assert X_tr.shape[0] + X_te.shape[0] == X.shape[0]
    assert y_tr.shape[0] + y_te.shape[0] == y.shape[0]
    
    # Fitting
    params = {'lamb': np.linspace(0.1, 2, 2), 'sigma': np.linspace(0.5, 2, 20), 'kernel': ['gaussian']}
    krr = GridSearchCV(KernelRidgeRegressor(), params)
#     krr = KernelRidgeRegressor()
    
    krr.fit(X_tr,y_tr)
    
    print(krr.best_params_)
    
    print(f"Accuracy on train set {name}: {krr.score(X_tr, y_tr):.2f}")
    print(f"Accuracy on test set {name} : {krr.score(X_te, y_te):.2f}")
    
    # Prediction on the new set
    X_eval = pd.read_csv(f'{data_folder}/Xte{name}_mat100.csv', sep = ' ', index_col=False, header=None).to_numpy()
    X_eval = (X_eval - mean)/std
    y_eval = krr.predict(X_eval)
    all_y_eval.append(y_eval)
    
all_y_eval = np.hstack(all_y_eval).reshape(-1)

{'kernel': 'gaussian', 'lamb': 0.1, 'sigma': 0.5789473684210527}
Accuracy on train set 0: 1.00
Accuracy on test set 0 : 0.57
{'kernel': 'gaussian', 'lamb': 0.1, 'sigma': 0.6578947368421053}
Accuracy on train set 1: 1.00
Accuracy on test set 1 : 0.59
{'kernel': 'gaussian', 'lamb': 0.1, 'sigma': 0.5}
Accuracy on train set 2: 1.00
Accuracy on test set 2 : 0.67


### 2. Kernel SVM

In [395]:
class KernelSVM(BaseEstimator, ClassifierMixin):
    
    def __init__(self, lamb=1., sigma=1., k=3, precomputed_kernel=None, normalize=False, kernel='gaussian'):
        """
        This class implements methods for fitting and predicting with a KernelRidgeRegressor used for classification 
        (by thresholding the value regressed). Any kernel can be used. 
        inputs:
        - lamb : the regularisation parameter 
        - sigma : the parameter of the Gaussian kernel (if Gaussian kernel selected)
        - kernel : the kernel we consider
        """
        self.lamb = lamb
        self.sigma = sigma
        self.k = k
        self.normalize = normalize
        self.kernel = kernel
        if self.kernel == 'gaussian':
            self.kernel_ = partial(Gaussian_kernel, sig=sigma)
        elif self.kernel == 'spectrum':
            self.kernel_ = partial(Spectrum_kernel, k=k)
        elif self.kernel == 'substring':
            if precomputed_kernel is not None:
                self.kernel_ = precomputed_kernel
            else:
                warnings.warn("Computing the kernel on the fly is computationnally heavy, you should probably precompute it.")
                self.kernel_ = partial(substring_kernel, k=k)
        else:
            raise NotImplementedError(f"Kernel {self.kernel} is not implemented yet")
    
    def fit(self, X, y):
        """
        inputs:
        - X (size: N_trxd): the points of the training set
        - y (size: N_trx1): the values of the classes
        """
        # We keep values of training in memory for prediction
        N_tr = X.shape[0]
        self.X_tr_ = np.copy(X)

        if self.kernel == 'gaussian':
            K = self.kernel_(X, X, sig=self.sigma)
        elif self.kernel == 'spectrum':
            K = self.kernel_(X, X, k=self.k[0])
            for i in range(len(self.k)-1):
                K+=self.kernel_(X, X, k=self.k[i+1])
        elif self.kernel == 'substring':
            K = self.kernel_(X, X)
            
        if self.normalize:
            self.tr_diag_ = np.copy(np.diag(K))
            K /= self.tr_diag_[:, None]
            K /= self.tr_diag_
            
        # Define QP and solve it with cvxpy
        alpha = cp.Variable(N_tr)
        objective = cp.Maximize(2*alpha.T@y - cp.quad_form(alpha, K))
        constraints = [0 <= cp.multiply(y,alpha), cp.multiply(y,alpha) <= 1/(2*self.lamb*N_tr)]
        prob = cp.Problem(objective, constraints)

        # The optimal objective value is returned by `prob.solve()`.
        result = prob.solve()
        # The optimal value for x is stored in `x.value`.
        self.alpha_ = alpha.value
        
        return self
        
    def predict(self, X):
        """
        inputs:
        - X (size N_texd): the points in R^d we want to classify
        output:
         - the predicted class for the associated y given the
        Linear Regression parameters
        """
        if self.kernel == 'gaussian':
            K_tr_te = self.kernel_(self.X_tr_, X, sig=self.sigma)
        elif self.kernel == 'spectrum':
            K_tr_te = self.kernel_(self.X_tr_, X, k=self.k[0])
            for i in range(len(self.k)-1):
                K_tr_te+=self.kernel_(self.X_tr_, X, k=self.k[i+1])
        elif self.kernel == 'substring':
            K_tr_te = self.kernel_(self.X_tr_, X)
            
        if self.normalize:
            K_tr_te /= np.power(self.tr_diag_[:, None], 2)
        
        return 2 * (self.alpha_.T@K_tr_te > 0).reshape(-1, ).astype("int") - 1
        
    def score(self, X, y):
        """
        inputs:
        - X (size N_texd): the points in R^d we want to classify
        - y (size N_tex1): the labels of the points
        """
        y_pred = self.predict(X)
        
        return np.sum(y_pred == y)/y.shape[0]

In [389]:
ar = np.random.rand(4, 4)
diag = np.copy(np.diag(ar))
ar /= diag **2
diag ** 2

array([0.7258116 , 0.0103188 , 0.79308337, 0.04436423])

#### Gaussian Kernel SVM

In [63]:
dim = 100
Nb_samples = 2000
prop_test = 0.2
lamb = 0.5
sigma = 1.2

all_y_eval = []

np.random.seed(1)
for name in [0, 1, 2]:
    # Data Processing
    X = pd.read_csv(f'{data_folder}/Xtr{name}_mat100.csv', sep = ' ', index_col=False, header=None).to_numpy()
    y = pd.read_csv(f'{data_folder}/Ytr{name}.csv')
    y = y["Bound"].to_numpy()
    y[y==0] = -1
    
    mean, std = X.mean(axis=0), X.std(axis=0)
    X = (X - mean)/std

    tr_indices = np.random.choice(Nb_samples, size=int((1-prop_test)*Nb_samples), replace=False)
    te_indices = [idx for idx in range(Nb_samples) if idx not in tr_indices]

    X_tr = X[tr_indices]
    X_te = X[te_indices]
    
    y_tr = y[tr_indices]
    y_te = y[te_indices]
    
    assert X_tr.shape[0] + X_te.shape[0] == X.shape[0]
    assert y_tr.shape[0] + y_te.shape[0] == y.shape[0]
    
    # Fitting
    params = {'lamb': np.logspace(-10., -7., 4), 'sigma': np.logspace(-1., 2., 4), 'kernel': ['gaussian']}
    ksvm = GridSearchCV(KernelSVM(), params)
#     krr = KernelRidgeRegressor()
    
    ksvm.fit(X_tr,y_tr)
    
    print(ksvm.best_params_)
    
    print(f"Accuracy on train set {name}: {ksvm.score(X_tr, y_tr):.2f}")
    print(f"Accuracy on test set {name} : {ksvm.score(X_te, y_te):.2f}")
    
    # Prediction on the new set
    X_eval = pd.read_csv(f'{data_folder}/Xte{name}_mat100.csv', sep = ' ', index_col=False, header=None).to_numpy()
    X_eval = (X_eval - mean)/std
    y_eval = ksvm.predict(X_eval)
    all_y_eval.append(y_eval)
    
all_y_eval = np.hstack(all_y_eval).reshape(-1)

{'kernel': 'gaussian', 'lamb': 1e-07, 'sigma': 100.0}
Accuracy on train set 0: 0.99
Accuracy on test set 0 : 0.62
{'kernel': 'gaussian', 'lamb': 1e-10, 'sigma': 1.0}
Accuracy on train set 1: 1.00
Accuracy on test set 1 : 0.60
{'kernel': 'gaussian', 'lamb': 1e-07, 'sigma': 100.0}
Accuracy on train set 2: 0.99
Accuracy on test set 2 : 0.72


#### Spectrum Kernel SVM

In [ ]:
## Kernel SVM with Spectrum kernel

dim = 100
Nb_samples = 2000
prop_test = 0.2
lamb = 0.5
sigma = 1.2
k = [4,5,6]

all_y_eval = []

np.random.seed(1)
for name in [0, 1, 2]:
    # Data Processing
    df = pd.read_csv(f'{data_folder}/Xtr{name}.csv')
    X = np.array(df['seq'])
    y = pd.read_csv(f'{data_folder}/Ytr{name}.csv')
    y = y["Bound"].to_numpy()
    y[y==0] = -1
    

    tr_indices = np.random.choice(Nb_samples, size=int((1-prop_test)*Nb_samples), replace=False)
    te_indices = [idx for idx in range(Nb_samples) if idx not in tr_indices]

    X_tr = X[tr_indices]
    X_te = X[te_indices]
    
    y_tr = y[tr_indices]
    y_te = y[te_indices]
    
    assert X_tr.shape[0] + X_te.shape[0] == X.shape[0]
    assert y_tr.shape[0] + y_te.shape[0] == y.shape[0]
    
    # Fitting
    # params = {'lamb': np.logspace(-10., -7., 4), 'k': np.array([3,4,5,6]), 'kernel': ['spectrum']}
    # ksvm = GridSearchCV(KernelSVM(), params)
#     krr = KernelRidgeRegressor()
    ksvm = KernelSVM(lamb = lamb, k=k, kernel='spectrum')
    
    ksvm.fit(X_tr,y_tr)
    
    #print(ksvm.best_params_)
    
    print(f"Accuracy on train set {name}: {ksvm.score(X_tr, y_tr):.2f}")
    print(f"Accuracy on test set {name} : {ksvm.score(X_te, y_te):.2f}")
    
    # Prediction on the new set
    X_eval = np.array(pd.read_csv(f'{data_folder}/Xte{name}.csv')['seq'])
    y_eval = ksvm.predict(X_eval)
    all_y_eval.append(y_eval)
    
all_y_eval = np.hstack(all_y_eval).reshape(-1)

Accuracy on train set 0: 0.72


#### Substring Kernel

In [397]:
# Kernel SVM with substring kernel

dim = 100
Nb_samples = 2000
prop_test = 0.2
lamb = 1e-6

all_y_eval = []

np.random.seed(1)
name = 0
# Data Processing
df = pd.read_csv(f'{data_folder}/Xtr{name}.csv')
X = np.array(df['seq'])
y = pd.read_csv(f'{data_folder}/Ytr{name}.csv')
y = y["Bound"].to_numpy()
y[y==0] = -1

df_eval = pd.read_csv(f'{data_folder}/Xte{name}.csv')
X_eval = np.array(df_eval['seq'])

precomputed_kernel = load_precomputed_kernel(df, df_eval, 
                                             f"{data_folder}/Ktrtr{name}_k=4_lambda=0.7.npy",
                                             f"{data_folder}/Ktreval{name}_k=4_lambda=0.7.npy")

tr_indices = np.random.choice(Nb_samples, size=int((1-prop_test)*Nb_samples), replace=False)
te_indices = [idx for idx in range(Nb_samples) if idx not in tr_indices]

X_tr = X[tr_indices]
X_te = X[te_indices]

y_tr = y[tr_indices]
y_te = y[te_indices]

assert X_tr.shape[0] + X_te.shape[0] == X.shape[0]
assert y_tr.shape[0] + y_te.shape[0] == y.shape[0]



# Fitting
# params = {'lamb': np.logspace(-10., -7., 4), 'k': np.array([3,4,5,6]), 'kernel': ['spectrum']}
# ksvm = GridSearchCV(KernelSVM(), params)
ksvm = KernelSVM(lamb=lamb, precomputed_kernel=precomputed_kernel, normalize=False, kernel='substring')

ksvm.fit(X_tr,y_tr)

# print(ksvm.best_params_)

print(f"Accuracy on train set {name}: {ksvm.score(X_tr, y_tr):.2f}")
print(f"Accuracy on test set {name} : {ksvm.score(X_te, y_te):.2f}")

# Prediction on the new set
y_eval = ksvm.predict(X_eval)
all_y_eval.append(y_eval)

all_y_eval = np.hstack(all_y_eval).reshape(-1)

Accuracy on train set 0: 0.72
Accuracy on test set 0 : 0.64


In [399]:
ids = np.arange(all_y_eval.shape[0])
filename = "results/submission_substring0_svm.csv"

# write_csv(ids, all_y_eval, filename)